In [1]:
import os
from dotenv import load_dotenv
load_dotenv()

True

## What is RAG (retrieval augmented generation)?
Basically, shoving lot of extra information in the prompt.

In [2]:
# Example 

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(
    prompt_template.format(
        context="Ajit has two sisters, Preeti and Sweta. Ajit is male.",
        question="Who is Preeti's bother?"         
    ))

print(response)

content="Ajit is Preeti's brother. He is also the brother of Sweta. This information is directly stated in the context provided." additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 119, 'total_tokens': 148, 'completion_time': 0.110944466, 'prompt_time': 0.016731196, 'queue_time': 0.047174454, 'total_time': 0.127675662}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None} id='run--196de901-e9c2-4f53-911e-97764e4ad013-0' usage_metadata={'input_tokens': 119, 'output_tokens': 29, 'total_tokens': 148}


In [3]:
print(response.content)

Ajit is Preeti's brother. He is also the brother of Sweta. This information is directly stated in the context provided.


>RAG is all about cleverly pushing is as much information in the context with minimum possible tokens

In [4]:
# little non-trival RAG example

# grab a novel
file = open("indianTales.txt")
text = file.read()

In [5]:
# lets try to talk with this book

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

# Prepare your prompt
prompt_template = """You are a novel reader. You are given collection of stories:
{collection_of_stories}
You are tasked to make a list of story titles in this collection. Write a short summary for each story in Hindi Language. Skip the story from the list, if the story is not provided in the text. 
"""

response = model.invoke(prompt_template.format(collection_of_stories = text[:len(text)//20]))

print(response)

content='यहाँ दी गई कहानियों की सूची है जिनके लिए मैंने हिंदी में एक संक्षिप्त सारांश लिखा है:\n\n1. **शेर और बगुला**: एक बगुला एक शेर की मदद करता है जिसके गले में एक हड्डी फंस जाती है, लेकिन जब बगुला मदद मांगता है तो शेर उसे धन्यवाद नहीं देता और कहता है कि वह उसका शिकार कर सकता था।\n\n2. **राजकुमार और राजकुमारी लबाम**: एक राजकुमार अपनी मां की सलाह के विरुद्ध जाकर राजकुमारी लबाम को ढूंढता है और उसे पाने के लिए कई चुनौतियों का सामना करता है। वह कई जानवरों से मिलता है, जिनमें से कुछ उसकी मदद करते हैं और कुछ उसके दुश्मन बन जाते हैं।\n\nकृपया ध्यान दें कि अन्य कहानियों के सारांश नहीं दिए गए हैं क्योंकि उनकी पूरी कहानी प्रदान नहीं की गई है।' additional_kwargs={} response_metadata={'token_usage': {'completion_tokens': 322, 'prompt_tokens': 5232, 'total_tokens': 5554, 'completion_time': 1.055895885, 'prompt_time': 0.398177272, 'queue_time': 0.062358288, 'total_time': 1.454073157}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_3f3b593e33', 'service_tier': 'on_demand', 'fin

In [6]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

यहाँ दी गई कहानियों की सूची है जिनके लिए मैंने हिंदी में एक संक्षिप्त सारांश लिखा है:

1. **शेर और बगुला**: एक बगुला एक शेर की मदद करता है जिसके गले में एक हड्डी फंस जाती है, लेकिन जब बगुला मदद मांगता है तो शेर उसे धन्यवाद नहीं देता और कहता है कि वह उसका शिकार कर सकता था।

2. **राजकुमार और राजकुमारी लबाम**: एक राजकुमार अपनी मां की सलाह के विरुद्ध जाकर राजकुमारी लबाम को ढूंढता है और उसे पाने के लिए कई चुनौतियों का सामना करता है। वह कई जानवरों से मिलता है, जिनमें से कुछ उसकी मदद करते हैं और कुछ उसके दुश्मन बन जाते हैं।

कृपया ध्यान दें कि अन्य कहानियों के सारांश नहीं दिए गए हैं क्योंकि उनकी पूरी कहानी प्रदान नहीं की गई है।

### Problem: Too much raw information in the context makes the prompt too long.
- Costly
- adds noise
### Solution: Use RAG
https://python.langchain.com/docs/tutorials/rag/
### To understand RAG, we need to understand Semantic Search
https://python.langchain.com/docs/tutorials/retrievers/

In [7]:
# load text using RELEVANT loader
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt")
docs = loader.load()

In [8]:
# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

Split given book into 563 sub-documents.


### Embedding

In [37]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Create a vector store
# (CLASSROOM DISCUSSION: What are vector stores? What do we make them?)
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


In [21]:
# document_ids

In [22]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

In [23]:
search_results

[(Document(id='529883b3-30e4-4ed2-93c3-d4723c7f1d11', metadata={'source': 'indianTales.txt', 'start_index': 110013}, page_content='goldsmith. Do not release him; and if you do, you shall surely repent\nof it one day or other." Thus advising, the hungry tiger went away\nwithout waiting for an answer.'),
  0.7302086830327078),
 (Document(id='561e4b7b-8902-490d-b58f-5882b0121227', metadata={'source': 'indianTales.txt', 'start_index': 30490}, page_content='When he heard of the demons the Raja\'s son was very sad. "What can I\ndo?" he said to himself. "How can I fight with these two demons?" Then\nhe thought of his tiger: and the tiger and his wife came to him and\nsaid, "Why are you so sad?" The Raja\'s son answered, "The king has\nordered me to fight with his two demons and kill them. How can I do\nthis?" "Do not be frightened," said the tiger. "Be happy. I and my\nwife will fight with them for you."\n\n[Illustration:]'),
  0.7208798323482593),
 (Document(id='63b309c0-9099-4a39-ad63-d9629

### What is RAG?
Retrieve using semantic search and dump the similar chunks in the context of the prompt.
LLM sees the question and retrieved docs in its prompt and generates tokens accordingly.

In [24]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

In [25]:
doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

goldsmith. Do not release him; and if you do, you shall surely repent
of it one day or other." Thus advising, the hungry tiger went away
without waiting for an answer.


When he heard of the demons the Raja's son was very sad. "What can I
do?" he said to himself. "How can I fight with these two demons?" Then
he thought of his tiger: and the tiger and his wife came to him and
said, "Why are you so sad?" The Raja's son answered, "The king has
ordered me to fight with his two demons and kill them. How can I do
this?" "Do not be frightened," said the tiger. "Be happy. I and my
wife will fight with them for you."

[Illustration:]


[Illustration:]

"What man hurt you that you roared so loud?" said the wife.

"No one hurt me," answered the husband; "but a Raja's son came and
took the thorn out of my foot."

"Where is he? Show him to me," said his wife.

"If you promise not to kill him, I will call him," said the tiger.

"I won't kill him; only let me see him," answered his wife.

Then the ti

In [26]:
# make the LLM read see the prompt, and analyse the retrieved document, and generate response

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))

In [27]:
# Better printing

from IPython.display import Markdown
Markdown(response.content)

There is no mention of a lion playing a role in the provided story context. The context primarily talks about a Raja's son, tigers, and other characters, but does not discuss a lion's role. I don't know the answer based on the given context.

# RAG Summary

In [58]:
# RAG summary

# Read a doc
from langchain_community.document_loaders import TextLoader
loader = TextLoader("indianTales.txt")
docs = loader.load()

# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

# embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Create a vector store
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


Split given book into 563 sub-documents.


In [59]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Lion in the story?",
    k = 10
)

doc_content = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

goldsmith. Do not release him; and if you do, you shall surely repent
of it one day or other." Thus advising, the hungry tiger went away
without waiting for an answer.


When he heard of the demons the Raja's son was very sad. "What can I
do?" he said to himself. "How can I fight with these two demons?" Then
he thought of his tiger: and the tiger and his wife came to him and
said, "Why are you so sad?" The Raja's son answered, "The king has
ordered me to fight with his two demons and kill them. How can I do
this?" "Do not be frightened," said the tiger. "Be happy. I and my
wife will fight with them for you."

[Illustration:]


[Illustration:]

"What man hurt you that you roared so loud?" said the wife.

"No one hurt me," answered the husband; "but a Raja's son came and
took the thorn out of my foot."

"Where is he? Show him to me," said his wife.

"If you promise not to kill him, I will call him," said the tiger.

"I won't kill him; only let me see him," answered his wife.

Then the ti

In [60]:
prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Lion in the story?"))


from IPython.display import Markdown
Markdown(response.content)

There is no mention of a lion playing a role in the provided story context. The context primarily talks about a tiger, a Raja's son, and other characters, but does not mention a lion's role. I don't know the answer based on the given context.

**SELF EXPERIMENTING**

In [61]:
#replicating the learnings from the notebook

In [62]:
# RAG summary

# Read a doc
from langchain_community.document_loaders import TextLoader
loader = TextLoader("Alice.txt")
docs = loader.load()

# Split document into small chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,  # chunk size (characters)
    chunk_overlap=200,  # chunk overlap (characters)
    add_start_index=True,  # track index in original document
)
all_splits = text_splitter.split_documents(docs)

print(f"Split given book into {len(all_splits)} sub-documents.")

# embedding
from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

# Create a vector store
from langchain_core.vectorstores import InMemoryVectorStore
vector_store = InMemoryVectorStore(embeddings)

# Adding documents to vector store
document_ids = vector_store.add_documents(documents=all_splits)


Split given book into 220 sub-documents.


In [63]:
# extract chunks which matches with your query

search_results = vector_store.similarity_search_with_score(
    "What is the role of Alice in the story?",
    k = 10
)

doc_content_Alice = "\n\n".join(doc.page_content+"\n"+"="*50+"\n" for (doc,score) in search_results)
print(doc_content)

goldsmith. Do not release him; and if you do, you shall surely repent
of it one day or other." Thus advising, the hungry tiger went away
without waiting for an answer.


When he heard of the demons the Raja's son was very sad. "What can I
do?" he said to himself. "How can I fight with these two demons?" Then
he thought of his tiger: and the tiger and his wife came to him and
said, "Why are you so sad?" The Raja's son answered, "The king has
ordered me to fight with his two demons and kill them. How can I do
this?" "Do not be frightened," said the tiger. "Be happy. I and my
wife will fight with them for you."

[Illustration:]


[Illustration:]

"What man hurt you that you roared so loud?" said the wife.

"No one hurt me," answered the husband; "but a Raja's son came and
took the thorn out of my foot."

"Where is he? Show him to me," said his wife.

"If you promise not to kill him, I will call him," said the tiger.

"I won't kill him; only let me see him," answered his wife.

Then the ti

In [64]:

from langchain.chat_models import init_chat_model
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

prompt_template = """You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.
Question: {question} 
Context: {context} 
Answer:"""

response = model.invoke(prompt_template.format(
    context=doc_content,
    question="What is the role of Alice in the story?"))

In [65]:
Markdown(response.content)

There is no mention of Alice in the provided context. I don't know the role of Alice in the story. The context appears to be a collection of folk tales or stories, but none of them mention a character named Alice.

In [66]:
##COMBINING THE ABOVE STEPS INTO A FUNCTION

In [67]:
from langchain.chat_models import init_chat_model
from IPython.display import Markdown

# Initialize model only once (outside function)
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

def answer_question(question: str, k: int = 10):
    """
    Retrieves context from the vector store and uses the model to answer the question.
    
    Args:
        question (str): The question you want to ask.
        k (int): Number of retrieved chunks. Default = 10.
    
    Returns:
        str: The model's answer.
    """

    # Step 1: Search relevant chunks
    search_results = vector_store.similarity_search_with_score(question, k=k)

    doc_content = "\n\n".join(
        doc.page_content + "\n" + "="*50 + "\n"
        for (doc, score) in search_results
    )

    # Step 2: Format prompt
    prompt_template = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.

    Question: {question} 
    Context: {context} 
    Answer:"""

    # Step 3: Get response
    response = model.invoke(prompt_template.format(
        context=doc_content,
        question=question
    ))

    # Step 4: Display as Markdown
    display(Markdown(response.content))

    return response.content

In [68]:
answer_question("What is the role of Alice in the story?")

Alice is the main character in the story, a curious and imaginative young girl who has many strange and wonderful adventures. She is the one narrating her dreams and experiences to her sister and interacting with various creatures in the story. Her role is that of a protagonist, driving the plot and exploring the fantastical world around her.

'Alice is the main character in the story, a curious and imaginative young girl who has many strange and wonderful adventures. She is the one narrating her dreams and experiences to her sister and interacting with various creatures in the story. Her role is that of a protagonist, driving the plot and exploring the fantastical world around her.'

In [69]:
answer_question("What is the role of Lion in the story?")

I don't know the role of the Lion in the story, as there is no mention of a Lion in the provided context. The context appears to be a collection of excerpts from various works, including "Alice in Wonderland" and other poems and stories. There is no information about a Lion's role in any of these stories.

'I don\'t know the role of the Lion in the story, as there is no mention of a Lion in the provided context. The context appears to be a collection of excerpts from various works, including "Alice in Wonderland" and other poems and stories. There is no information about a Lion\'s role in any of these stories.'

Making the Function that

1. Loads a text file.

2. Splits it into chunks.

3. Embeds and stores it in a vector store.

4. Lets you ask a question.

5. Returns + prints the model’s answer.

In [71]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.vectorstores import InMemoryVectorStore
from langchain.chat_models import init_chat_model
from IPython.display import Markdown

# Initialize model only once
model = init_chat_model("llama-3.3-70b-versatile", model_provider="groq")

def Rag_func_1(file_name: str, question: str, k: int = 10):
    """
    Loads a text file, creates embeddings, stores in a vector store,
    retrieves relevant chunks, and answers the question.

    Args:
        file_name (str): Path to the text file.
        question (str): The question to ask.
        k (int): Number of retrieved chunks. Default = 10.

    Returns:
        str: The model's answer.
    """

    # Step 1: Load document
    loader = TextLoader(file_name)
    docs = loader.load()

    # Step 2: Split into chunks
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=1000,
        chunk_overlap=200,
        add_start_index=True,
    )
    all_splits = text_splitter.split_documents(docs)

    print(f"📖 Split '{file_name}' into {len(all_splits)} sub-documents.")

    # Step 3: Embedding & vector store
    embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
    vector_store = InMemoryVectorStore(embeddings)
    vector_store.add_documents(documents=all_splits)

    # Step 4: Retrieve context
    search_results = vector_store.similarity_search_with_score(question, k=k)
    doc_content = "\n\n".join(
        doc.page_content + "\n" + "="*50 + "\n"
        for (doc, score) in search_results
    )

    # Step 5: Prompt template
    prompt_template = """You are an assistant for question-answering tasks. 
    Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know. 
    Use three sentences maximum and keep the answer concise.

    Question: {question} 
    Context: {context} 
    Answer:"""

    # Step 6: Query the model
    response = model.invoke(prompt_template.format(
        context=doc_content,
        question=question
    ))

    # Step 7: Show result
    display(Markdown(response.content))

    return response.content


In [72]:
Rag_func_1("Little Women.txt", "What is the role of Laurie in the story?")

📖 Split 'Little Women.txt' into 145 sub-documents.


Laurie is a character in the story who is introduced as the "Laurence boy" and is a neighbor of the main characters. He is a polite and handsome young man who befriends Jo and her sisters, and they have a merry time together. Laurie is also shown to be well-traveled and educated, having spent time abroad and speaking French.

'Laurie is a character in the story who is introduced as the "Laurence boy" and is a neighbor of the main characters. He is a polite and handsome young man who befriends Jo and her sisters, and they have a merry time together. Laurie is also shown to be well-traveled and educated, having spent time abroad and speaking French.'

.
.
.
.
.

With this function, we can imagine having an **infinite library of books**.  
We can load any file from that library and ask any question to instantly get answers from it.

.
.
.
.
.